In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import sys 
installation_path = "/content/drive/MyDrive/Colab_Installations_V2"
# The path is being modified so that everything installed in the installation path can now be used without re-installing (in this case, I just need biopython)
sys.path.append(installation_path)
protein_mpnn_path = "/content/drive/MyDrive/Protein_MPNN_Digging/ProteinMPNN/vanilla_proteinmpnn"
sys.path.insert(0,protein_mpnn_path)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Protein_MPNN_Digging

/content/drive/MyDrive/Protein_MPNN_Digging


In [4]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm
from Bio.PDB.Polypeptide import *
from Bio.PDB import *

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import Ridge
from scipy.stats import entropy
from scipy.special import softmax
from scipy.special import kl_div
from scipy.stats import pearsonr
from sklearn.svm import SVR
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
with open("S_2648_pmppn_info_dict.pickle","rb") as f:
    S_2648_two_level_dict = pickle.load(f)
with open("S_921_pmppn_info_dict.pickle","rb") as f:
    S_921_two_level_dict = pickle.load(f)
with open("S_669_pmppn_info_dict.pickle","rb") as f:
    S_669_two_level_dict = pickle.load(f)
with open("Ssym_pmppn_info_dict.pickle","rb") as f:
    Ssym_two_level_dict = pickle.load(f)

In [ ]:
list_two_level_dict = [S_2648_two_level_dict,S_921_two_level_dict,S_669_two_level_dict,Ssym_two_level_dict]
list_dataset_names = ["S_2648","S_921","S_669","Ssym"]

In [ ]:
list_mapping_dict = []
list_proteins_to_skip = []

for name in list_dataset_names:
    mapping_dict = {}
    pdbDirectory = f"/content/drive/MyDrive/ACCRE_PyRun_Setup/{name}_PDB_Files"
    parser = PDBParser(QUIET=True)
    # some proteins need to be skipped for now due to ICODE related discrapency
    proteins_to_skip = []

    for filename in tqdm(os.listdir(pdbDirectory)):
        filepath = os.path.join(pdbDirectory,filename)
        structure = parser.get_structure(id=filename.split(".")[0],file=filepath)
        model = structure[0]
        inner_dict = {}
        outer_key = filename.split(".")[0]
        skip_flag = False
        # single chain-assumption in action again
        for chain in model:
            for i,residue in enumerate(chain):
                inner_key = f"{three_to_one(residue.get_resname())}{residue.get_id()[1]}"
                if inner_key not in inner_dict:
                    inner_dict[inner_key] = i
                else:
                    # For "2immA:N31" and "1lveA:S27", I have been fucked
                    # Need to think whether this will effect other positions or I can just avoid these two-protein related mutations for now?
                    # Let me just avoid these two proteins for now
                    print("YOU HAVE JUST BEEN FUCKED BY ICODE")
                    print(f"{outer_key}:{inner_key}")
                    skip_flag = True
        # The ICODE related problematic proteins will not be considered for now
        if not skip_flag:
            mapping_dict[outer_key] = inner_dict
        else:
            proteins_to_skip.append(outer_key)
    list_proteins_to_skip.append(proteins_to_skip)
    list_mapping_dict.append(mapping_dict)

  0%|          | 0/131 [00:00<?, ?it/s]

YOU HAVE JUST BEEN FUCKED BY ICODE
1lveA:S27
YOU HAVE JUST BEEN FUCKED BY ICODE
1lveA:S27
YOU HAVE JUST BEEN FUCKED BY ICODE
2immA:N31
YOU HAVE JUST BEEN FUCKED BY ICODE
2immA:N31


  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/357 [00:00<?, ?it/s]

In [ ]:
# let me get PSSM values and do some comparison quickly
# getting the PSSM extraction functions from my custom model data processing scripts
from string import ascii_uppercase

# In rare cases, some PDB files number chains with 1,2,3 instead of A,B,C
def convertChainFromAlphabetToNumber(alphabet):
    mappingDict = {ch:(idx+1) for idx,ch in enumerate(ascii_uppercase)}
    return str(mappingDict[alphabet])

# Before executing this function, the PSSM files with naming format "pdbIdchain.pssm" needs to be stored
# in the pssm_dir
def returnPSSMArray(pdbIdPlusChain,pssm_dir="train_pssm_dir",convert_upper = False):
#     Currently, assuming that the pssm file names contain pdbId in upper case
    if convert_upper:
        fileName = pdbIdPlusChain.upper() + ".pssm"
    else:
        fileName = pdbIdPlusChain + ".pssm"
    try:
        fullPath = os.path.join(pssm_dir,fileName)
        f = open(fullPath)
    except:
        fileName = pdbIdPlusChain[0:4].upper() + str(convertChainFromAlphabetToNumber(pdbIdPlusChain[4])) + ".pssm" 
        fullPath = os.path.join(pssm_dir,fileName)
        f = open(fullPath)
        
# #     all the target lines in the PSSM files have (2+20+20+2=44) strings after line.split()
    target_lines = [line.split() for line in f.readlines() if (len(line.split()))==44]
    number_of_residues = len(target_lines)
    
    pssm_features = np.zeros((number_of_residues,20))

    for idx,line in enumerate(target_lines):
        pssm_features[idx,:] = line[2:22]

    f.close()
    
    return pssm_features

# This function also seems necessary for extracting the two pssm values
# Must review the three pssm feature functions (this one and the two above) later
# These functions seem to be taking up a lot of time....must review
def returnPSSMMapping(residue):
    pssm_letter_to_index_dict = {"A" : 0,   
    "R" : 1,
    "N" : 2,
    "D" : 3,
    "C" : 4,
    "Q" : 5,
    "E" : 6,
    "G" : 7,
    "H" : 8,
    "I" : 9,
    "L" : 10,
    "K" : 11,
    "M" : 12,
    "F" : 13,
    "P" : 14,
    "S" : 15,
    "T" : 16,
    "W" : 17,
    "Y" : 18,
    "V" : 19}

    return pssm_letter_to_index_dict[residue]

for mapping_dict,name,two_level_dict,proteins_to_skip in zip(list_mapping_dict,list_dataset_names,list_two_level_dict,list_proteins_to_skip):
    # I will add PSSM values to the two-level dictionary for places where log_prob is available
    pssmDirectory = f"/content/drive/MyDrive/ACCRE_PyRun_Setup/{name}_pssm_dir"
    for prot,muts in two_level_dict.items():
        if prot not in proteins_to_skip:
            try:
                cur_map_dict = mapping_dict[prot]
            except:
                continue
            for mut in muts:
                # only fetching those mutations that have corresponding log-probabilities calculated and saved as values of "log_prob" key
                if "log_prob" in mut:
                    wild = mut["mut"][0] 
                    alternate = mut["mut"][-1]
                    sequence_index_of_mutation = cur_map_dict[mut["mut"][0:-1]]
                    pdbId = prot[0:-1]
                    mutChain = prot[-1]
                    pssm_array = returnPSSMArray(pdbId + mutChain,pssm_dir=pssmDirectory,convert_upper = False)
                    position_pssm = pssm_array[sequence_index_of_mutation]
                    wild_pssm = position_pssm[returnPSSMMapping(wild)] 
                    alternate_pssm = position_pssm[returnPSSMMapping(alternate)]
                    mut["wild_pssm"] = wild_pssm
                    mut["alternate_pssm"] = alternate_pssm

In [ ]:
# Create the six numpy arrays here, and then perform training on the next cell
X_2648 = []
y_2648 = []
X_921 = []
y_921 = []
X_669 = []
y_669 = []
X_Ssym = []
y_Ssym = []

# This list will correspond to the "list_mapping_dict,list_dataset_names,list_two_level_dict,list_proteins_to_skip" lists in the above cells, 
# and hold (X,y) tuples corresponding to the three datasets in the same order as those lists
# currently, I intend to train, and test the model on "mut_wild_predictions","entropy_predictions", "neighbor_energy_change_predictions", "neighbor_forward_KL_predictions", 
# and "neighbor_backward_KL_predictions" 
# fourth and fifth features can probably be reduced to one feature by (0.5*fourth_feature+0.5*fifth_feature) since both of them are extremely correlated
list_X = [X_2648,X_921,X_669,X_Ssym]
list_y = [y_2648,y_921,y_669,y_Ssym]

#######
alpha_tok = "ACDEFGHIKLMNPQRSTVWYX"
aa_1_N = {a:n for n,a in enumerate(alpha_tok)}
for X,y,two_level_dict,mapping_dict,proteins_to_skip in zip(list_X,list_y,list_two_level_dict,list_mapping_dict,list_proteins_to_skip):
    for prot,muts in two_level_dict.items():
        if prot not in proteins_to_skip:
            try:
                cur_map_dict = mapping_dict[prot]
            except:
                continue
            for mut in muts:
                 # This "cur_X" will hold the features for the current mutation under-processing in the current inner loop iteration
                cur_X = []
                wild = mut["mut"][0] 
                alternate = mut["mut"][-1]
                y.append(mut["ddg"])
                sequence_index_of_mutation = cur_map_dict[mut["mut"][0:-1]]
                position_log_probabilities = mut["log_prob"][0,sequence_index_of_mutation,:]
                
                ### first feature addition
                mut_wild_prediction = (-1*position_log_probabilities[aa_1_N[alternate]]) - (-1*position_log_probabilities[aa_1_N[wild]])
                cur_X.append(mut_wild_prediction)
                ### first feature addition
                
                ### second feature addition
                entropy_prediction = entropy(np.exp(position_log_probabilities))
                cur_X.append(-1*entropy_prediction)
                ### second feature addition

                # These 0-based neighbor indices will be used for extracting the log-probabilities corresponding to the neighbor positions
                n_indices= mut["top_15_neighbor_indices"]
                # The neighbor weights will be used here for multiplying 
                n_weights = mut["top_15_attention_weights"].reshape(-1,1)

                # Now, calculate neighbor energy changes, and then weighted sum them after extracting specific log_probabilities
                # for mutant center, and wildtype center impacted versions for the top neighbor positions
                neighbor_w_log_probabilities = mut["w_n_log_prob"]
                neighbor_m_log_probabilities = mut["m_n_log_prob"]
                neighbor_amino_a_identities = mut["neighbor_aa_identities"]
                # The "local_neighbor_log_prob_vals" will be a list of negative log-probability differences(a.k.a. energy differences)
                local_neighbor_log_prob_vals = []
                local_neighbor_forward_KL_vals = []
                local_neighbor_backward_KL_vals = []
                # For example, selecting the numbers from the first 5 iterations of this loop will give neighbor energy change corresponding to the first 5 neighbors
                for neighbor_w, neighbor_m, neighbor_aa  in zip(neighbor_w_log_probabilities,neighbor_m_log_probabilities,neighbor_amino_a_identities):
                    # get the amino acid identity for the neighbor position, run it through the mapping dictionary, get the log probabilities from those positions,
                    # "neighbor_w" and "neighbor_m" arrays will directly give the log probabilities that need to be substracted to get the energy (put (-1) before thoese numbers?...think a bit)
                    neighbor_index = aa_1_N[neighbor_aa]
                    local_neighbor_log_prob_vals.append((-1*neighbor_m[neighbor_index])-(-1*neighbor_w[neighbor_index]))
                    # summing the output of "kl_div", because one number comes for every positions in the currently processing neighbor distribution,
                    # and I want to take the total deviation in that distribution 
                    local_neighbor_forward_KL_vals.append(kl_div(np.exp(neighbor_w),np.exp(neighbor_m)).sum())
                    local_neighbor_backward_KL_vals.append(kl_div(np.exp(neighbor_m),np.exp(neighbor_w)).sum())

                ### third feature addition
                weighted_neighbor_entropy_prediction = (np.array(local_neighbor_log_prob_vals[0:15])*softmax(n_weights[0:15])).sum()
                cur_X.append(weighted_neighbor_entropy_prediction)
                ### third feature addition

                ### fourth feature addition
                weighted_neighbor_forward_KL_prediction = (np.array(local_neighbor_forward_KL_vals[0:15])*softmax(n_weights[0:15])).sum()
                cur_X.append(weighted_neighbor_forward_KL_prediction)
                ### fourth feature addition

                ### fifth feature addition
                weighted_neighbor_backward_KL_prediction = (np.array(local_neighbor_backward_KL_vals[0:15])*softmax(n_weights[0:15])).sum()
                cur_X.append(weighted_neighbor_backward_KL_prediction)
                ### fifth feature addition

                ### adding the two PSSM features
                cur_X.append(mut["wild_pssm"]-mut["alternate_pssm"])
                ### adding the two PSSM features
                X.append(np.array(cur_X))

In [ ]:
S_2648_X = np.array(list_X[0])
S_2648_y = np.array(list_y[0])
S_921_X = np.array(list_X[1])
S_921_y = np.array(list_y[1])
S_669_X = np.array(list_X[2])
S_669_y = np.array(list_y[2])
# Flipping the signs of the experimental DDG values in S_669_y to get coherent correlations (positive and higher is better)
S_669_y = S_669_y * (-1)
Ssym_X = np.array(list_X[3])
Ssym_y = np.array(list_y[3])

In [86]:
# quick dirty testing to see whether dictionary saving of the features have worked, or not
import pickle
with open("Ssym_full_feature_dict.pickle","rb") as f:
    t_dict = pickle.load(f)
t_feat = []
t_val = []
t_skip = list_proteins_to_skip[-1]
for prot,muts in t_dict.items():
    if prot not in t_skip:
        # we are now traversing the list "muts", which has information regarding all the mutations on a specific protein, "prot" 
        for mut in muts:
            t_feat.append(mut["ddg"])
            t_val.append(mut["alternate_pssm"]-mut["wild_pssm"])
print(pearsonr(np.array(t_val),np.array(t_feat)))

(-0.25485992422573917, 1.7969755175311432e-06)


In [74]:
}print("S_921 single-feature correlations at a glance")
print(pearsonr(S_921_y,S_921_X[:,0]))
print(pearsonr(S_921_y,S_921_X[:,1]))
print(pearsonr(S_921_y,S_921_X[:,2]))
print(pearsonr(S_921_y,S_921_X[:,3]))
print(pearsonr(S_921_y,S_921_X[:,4]))
print(pearsonr(S_921_y,S_921_X[:,-1]))
print(".............................................")

print("S_669 single-feature correlations at a glance")
print(pearsonr(S_669_y,S_669_X[:,0]))
print(pearsonr(S_669_y,S_669_X[:,1]))
print(pearsonr(S_669_y,S_669_X[:,2]))
print(pearsonr(S_669_y,S_669_X[:,3]))
print(pearsonr(S_669_y,S_669_X[:,4]))
print(pearsonr(S_669_y,S_669_X[:,-1]))
print(".............................................")

print("S_2648 single-feature correlations at a glance")
print(pearsonr(S_2648_y,S_2648_X[:,0]))
print(pearsonr(S_2648_y,S_2648_X[:,1]))
print(pearsonr(S_2648_y,S_2648_X[:,2]))
print(pearsonr(S_2648_y,S_2648_X[:,3]))
print(pearsonr(S_2648_y,S_2648_X[:,4]))
print(pearsonr(S_2648_y,S_2648_X[:,-1]))
print(".............................................")

print("Ssym single-feature correlations at a glance")
print(pearsonr(Ssym_y,Ssym_X[:,0]))
print(pearsonr(Ssym_y,Ssym_X[:,1]))
print(pearsonr(Ssym_y,Ssym_X[:,2]))
print(pearsonr(Ssym_y,Ssym_X[:,3]))
print(pearsonr(Ssym_y,Ssym_X[:,4]))
print(pearsonr(Ssym_y,Ssym_X[:,-1]))
print(".............................................")

S_921 single-feature correlations at a glance
(0.6441949275101646, 4.197628384942669e-109)
(0.2558421609543604, 3.1365846164886655e-15)
(0.437681171507952, 2.1878261091647976e-44)
(0.3121729665217345, 2.884837358775896e-22)
(0.3927292979111014, 2.4800280644811723e-35)
(0.5145587134119901, 2.1316560964554222e-63)
.............................................
S_669 single-feature correlations at a glance
(0.33765558573837523, 1.772522028540666e-18)
(0.26048284702580865, 2.3531867527057347e-11)
(0.18795423751117238, 1.7456892815458912e-06)
(0.33503660186553375, 3.3591731770954463e-18)
(0.3546264904588711, 2.4222666946646233e-20)
(0.1760175004428737, 7.741516443498937e-06)
.............................................
S_2648 single-feature correlations at a glance
(0.5046657755393072, 2.4008447333492008e-169)
(0.3517800116434, 3.516441939464636e-77)
(0.304002185256651, 3.8146414892394183e-57)
(0.21403439183637213, 1.5872556482974816e-28)
(0.25260740505184043, 2.0101930704841394e-39)
(0.275

In [ ]:
# Let us try to add some reverse mutants quickly
S_2648_X_aug = []
S_2648_y_aug = []
for X,y in zip(S_2648_X,S_2648_y):
    S_2648_X_aug.append(X)
    S_2648_X_aug.append(-1*X)
    S_2648_y_aug.append(y)
    S_2648_y_aug.append(-1*y)
S_2648_X_aug = np.array(S_2648_X_aug)
S_2648_y_aug = np.array(S_2648_y_aug)

S_921_X_aug = []
S_921_y_aug = []
for X,y in zip(S_921_X,S_921_y):
    S_921_X_aug.append(X)
    S_921_X_aug.append(-1*X)
    S_921_y_aug.append(y)
    S_921_y_aug.append(-1*y)
S_921_X_aug = np.array(S_921_X_aug)
S_921_y_aug = np.array(S_921_y_aug)

S_669_X_aug = []
S_669_y_aug = []
for X,y in zip(S_669_X,S_669_y):
    S_669_X_aug.append(X)
    S_669_X_aug.append(-1*X)
    S_669_y_aug.append(y)
    S_669_y_aug.append(-1*y)
S_669_X_aug = np.array(S_669_X_aug)
S_669_y_aug = np.array(S_669_y_aug)

Ssym_X_aug = []
Ssym_y_aug = []
for X,y in zip(Ssym_X,Ssym_y):
    Ssym_X_aug.append(X)
    Ssym_X_aug.append(-1*X)
    Ssym_y_aug.append(y)
    Ssym_y_aug.append(-1*y)
Ssym_X_aug = np.array(Ssym_X_aug)
Ssym_y_aug = np.array(Ssym_y_aug)

In [ ]:
# Now, traing linear regression on S_2648, and test on S_921 and S_669
# the setting reg = Ridge(alpha=0.2,normalize=True).fit(S_2648_X_aug[:,[0,2,4,5]], S_2648_y_aug) seems quite good
# Ridge(alpha=0.25,normalize=True).fit(S_2648_X_aug[:,[0,2,4,5]], S_2648_y_aug) is also good
# Ridge(alpha=0,normalize=True) is also not bad
reg = Ridge(alpha=0,normalize=True).fit(S_669_X_aug[:,[0,2,4,5]], S_669_y_aug)
print(pearsonr(S_2648_y,reg.predict(S_2648_X[:,[0,2,4,5]])))
print(pearsonr(S_921_y_aug,reg.predict(S_921_X_aug[:,[0,2,4,5]])))
print(pearsonr(S_669_y_aug,reg.predict(S_669_X_aug[:,[0,2,4,5]])))
print(pearsonr(Ssym_y_aug,reg.predict(Ssym_X_aug[:,[0,2,4,5]])))

(0.4832532690335158, 2.0398666299936044e-153)
(0.724232540206807, 2.3830207176529067e-299)
(0.6193673565561999, 4.921820803949098e-136)
(0.6744746488870841, 6.154354491219317e-92)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  FutureWarning,


In [ ]:
print(reg.coef_)
print(reg.intercept_)

[0.17360795 0.00180166 0.78120299 0.03388041]
0.0


In [ ]:
sv = SVR(C=50000)
normalizer = Normalizer()
normalizer.fit(S_2648_X_aug[:,[0,2,4,5]])
sv.fit(normalizer.transform(S_2648_X_aug[:,[0,2,4,5]]),S_2648_y_aug)
print(pearsonr(S_2648_y,sv.predict(normalizer.transform(S_2648_X[:,[0,2,4,5]]))))
print(pearsonr(S_921_y_aug,sv.predict(normalizer.transform(S_921_X_aug[:,[0,2,4,5]]))))
print(pearsonr(S_669_y_aug,sv.predict(normalizer.transform(S_669_X_aug[:,[0,2,4,5]]))))
print(pearsonr(Ssym_y_aug,sv.predict(normalizer.transform(Ssym_X_aug[:,[0,2,4,5]]))))

(0.49341056142252027, 7.846291916351464e-161)
(0.6130594343744621, 1.4211506158078185e-190)
(0.4967697799969245, 1.783272204920996e-80)
(0.6531245627269202, 2.0437393473422155e-84)


In [ ]:
gbr = GradientBoostingRegressor(min_samples_split=70)
gbr.fit(Ssym_X_aug[:,[0,2,4,5]],Ssym_y_aug)
print(pearsonr(S_2648_y,gbr.predict(S_2648_X[:,[0,2,4,5]])))
print(pearsonr(S_921_y_aug,gbr.predict(S_921_X_aug[:,[0,2,4,5]])))
print(pearsonr(S_669_y_aug,gbr.predict(S_669_X_aug[:,[0,2,4,5]])))
print(pearsonr(Ssym_y_aug,gbr.predict(Ssym_X_aug[:,[0,2,4,5]])))

(0.5047609507610759, 2.0276729097383117e-169)
(0.7552747118628999, 0.0)
(0.5483764179940949, 4.502626762627943e-101)
(0.8645156870258635, 6.096424897148414e-206)


In [ ]:
gbr.feature_importances_

array([0.75844459, 0.08660043, 0.04745311, 0.10750187])

In [ ]:
# Now, traing linear regression on S_2648, and test on S_921 and S_669
rf = RandomForestRegressor(min_samples_split=32).fit(S_2648_X_aug[:,[0,2,4,5]],S_2648_y_aug)
print(pearsonr(S_2648_y,rf.predict(S_2648_X[:,[0,2,4,5]])))
print(pearsonr(S_921_y_aug,rf.predict(S_921_X_aug[:,[0,2,4,5]])))
print(pearsonr(S_669_y_aug,rf.predict(S_669_X_aug[:,[0,2,4,5]])))

(0.7673360988690586, 0.0)
(0.75238603932414, 0.0)
(0.579911151448481, 1.5225141857589223e-115)


In [ ]:
rf.feature_importances_

array([0.65339947, 0.07879502, 0.19869529, 0.06911022])